#### Names of people in the group

Please write the names of the people in your group in the next cell.

Magnus Sagmo

Elizabeth Pan

In [0]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

In [0]:
# Loading modules that we need
from pyspark.sql.dataframe import DataFrame
from collections import Counter
from pyspark.sql.functions import desc, avg, col, split, regexp_replace, explode, monotonically_increasing_id
from pyspark.sql import Row

In [0]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame 
def load_df(table_name: "name of the table to load") -> DataFrame:
    return spark.read.parquet(table_name)

users_df = load_df("/user/hive/warehouse/users")
posts_df = load_df("/user/hive/warehouse/posts")

#### Subtask 1: implementing two functions
Implement these two functions:
1. 'compute_pearsons_r' that receives a DataFrame and two column names and returns the [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) between values of two columns;
2. 'make_tag_graph' that in the input receives the DataFrame containing the records related to 'questions' and returns a DataFrame with two columns 'u' and 'v'; the record for row i from the resulting DataFrame is a tuple (u_i, v_i). u_i and v_j are distinct tags and have appeared together for a question.

Please note that you should implement the 'compute_pearsons_r' yourself, so you should not use the 'DataFrame.stat.corr' method. Nevertheless, you can use 'DataFrame.stat.corr' to verify the correctness of your implementation.

In [0]:
from math import sqrt

def compute_sd(df: "DataFrame", col_n: "Column"):
  return sqrt(
    df.select(avg(col(col_n) * col(col_n))).collect()[0][0] - (df.select(avg(col(col_n))).collect()[0][0] * df.select(avg(col(col_n))).collect()[0][0])
  )

def compute_pearsons_r(df: "a DataFrame", col1: "name of column A", col2: "name of column B") -> float:
  # computing the individual standard deviations
  sd_1 = compute_sd(df, col1)
  sd_2 = compute_sd(df, col2)
  
  # computing the standard deviation between the columns
  sd_1_2 = df.select(avg(col(col1) * col(col2))).collect()[0][0] - (df.select(avg(col(col1))).collect()[0][0] * df.select(avg(col(col2))).collect()[0][0])
  
  # computing the pearson correlation coefficient. 
  return sd_1_2/(sd_1 * sd_2)


 # from piazza
def f(tag_list):
  cooccurences = set()
  if len(tag_list) == 1:
    return [(tag_list[0], tag_list[0])]
  else:
    for i in tag_list:
      for j in tag_list:
        if i != j and (j, i) not in cooccurences and i != "" and j != "":
          cooccurences.add((i, j))
          cooccurences.add((j, i))
    return list(cooccurences) 
          
def make_tag_graph(df: "DataFrame containing question data") -> DataFrame:

  rdd = df.select("Tags").filter(col("Tags").contains("><")).rdd

  cooccurences = []
  
  for row in rdd.collect():
    altered_row = row["Tags"].replace("<", "").split(">")
    altered_row.sort()
    tags = f(altered_row)
    for tag in tags:
      cooccurences.append(tag)

  R = Row("u", "v")  
  result_df = sc.parallelize([R(*r) for r in cooccurences]).toDF()

  return result_df

In [0]:
# Imprting GraphFrames graph library; make sure you have GraphFrames installed on the cluster
from graphframes import *

#### Subtask 2: implementing three functions
Impelment these three functions:
1. 'get_nodes' that, given the result from execution of 'make_tag_graph', returns a DataFrame with one column named 'id' that includes the tags that have appeared in the tag graph;
2. 'get_edges' that, given the result from execution of 'make_tag_graph', returns a DataFrame with two columns 'src' and 'dst' where 'src' is the source node and 'dst' is the destination node.
3. 'compute_pagerank' that receives a GraphFrames graph object in the input and computes the PageRank for nodes in the graph and returns the result as a DataFrame with two columns named 'id' and 'pagerank'; the rows in the in the resulting DataFrame should be sorted by the values of 'pagerank' column.

Note that the term 'tag graph' in this context refers to the DataFrame reuturned by executing 'make_tag_graph'. Furthermore, 'src' and 'dst' are distinct, so 'src' != 'dst'.

In [0]:
def get_nodes(df: "DataFrame of the tag graph") -> DataFrame:
  data = df.select("u").withColumnRenamed("u", "id").distinct()
  return data

def get_edges(df: "DataFrame of the tag graph") -> DataFrame:
  data = df.withColumnRenamed("u", "src").withColumnRenamed("v", "dst")
  return data

def compute_pagerank(graph: "a Graphframes graph") -> DataFrame:
  results = graph.pageRank(tol=0.1)
  results = results.vertices.select("id", "pagerank").orderBy(col("pagerank").desc())
  return results
  

In [0]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

#### Subtask 3: validating the implementation by running the tests

Run the cell below and make sure that all the tests run successfully.

In [0]:
%%unittest_main
class TestTask3(unittest.TestCase):
  
  error_threshold = 0.03
  
  def test_corr1(self):
    # Pearson correlation coefficient between 'user reputation' and 'upvotes' received by users
    result = compute_pearsons_r(users_df, "Reputation", "UpVotes")
    self.assertLessEqual(abs(result-0.5218138310114108), self.error_threshold)
    print(result)
  
  def test_corr2(self):
    # Pearson correlation coefficient between 'user reputation' and 'downvotes' received by users
    result = compute_pearsons_r(users_df, "Reputation", "DownVotes")
    self.assertLessEqual(abs(result-0.1473558141546844), self.error_threshold)
    print(result)

  def test_corr3(self):
    # Pearson correlation coefficient between 'question score' and the 'number of answers' it received
    result = compute_pearsons_r(posts_df[posts_df["PostTypeId"] == 1], "Score", "AnswerCount")
    self.assertLessEqual(abs(result-0.47855272641249674), self.error_threshold)
    print(result)
    
  def test_make_tag_graph(self):
    result = make_tag_graph(df=posts_df[posts_df["PostTypeId"] == 1])
    self.assertIsInstance(result, DataFrame)
    
    coulmn_names = Counter(map(str.lower, ['u', 'v']))
    self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)), "Missing column(s) or column name mismatch")
    
    display(result)
    
    self.assertEqual(result.count(), 228830)
    
  def test_get_nodes(self):
    result = make_tag_graph(df=posts_df[posts_df["PostTypeId"] == 1])
    n = get_nodes(result)
    self.assertEqual(n.count(), 638)
    n.show()

  def test_get_edges(self):
    result = make_tag_graph(df=posts_df[posts_df["PostTypeId"] == 1])
    e = get_edges(result)
    
    coulmn_names = Counter(map(str.lower, ['src', 'dst']))
    self.assertCountEqual(coulmn_names, Counter(map(str.lower, e.columns)), "Missing column(s) or column name mismatch")
    
    self.assertEqual(e.count(), 225290)
    e.show()
    
  def test_compute_pagerank(self):
    result = make_tag_graph(df=posts_df[posts_df["PostTypeId"] == 1])
    n = get_nodes(result)
    e = get_edges(result)
    g = GraphFrame(n, e)
    ranks = compute_pagerank(g)
    self.assertEqual(ranks.first()[0], 'machine-learning')
    ranks.show()

u,v
open-source,education
education,open-source
definitions,data-mining
data-mining,definitions
bigdata,machine-learning
machine-learning,bigdata
bigdata,libsvm
machine-learning,libsvm
libsvm,bigdata
libsvm,machine-learning


#### Subtask 4: answering to questions about Spark related concepts

Please write a short description for the terms below---one to two short paragraphs for each term. Don't copy-paste; instead, write your own understanding.

1. What do the terms 'User-Defined Functions (UDFs)', 'Data Locality', 'Bucketing', 'Distributed Filesystem' mean in the context of Spark?

Write your descriptions in the next cell.

##### User-Defined Functions
_User-Defined Functions_, or UDFs, allow a user to define it's own transformation functions in addition to Spark's built-in _Standard Functions_. They are created by writing a function with regular Python syntax, and wrapping it with Spark's ```udf``` function. 

If one would wnat to tranform a regular string into an HTML ```<p>```-tag, one could write a function that takes the content of one column, and adds it to a new column named _Paragraph_, with an opening and closing ```<p>```-tag before and after the original content. 

[sparkbyexamples.com](https://sparkbyexamples.com/pyspark/pyspark-udf-user-defined-function/)

##### Data Locality
Due to the vast (possible) size size of the data being handled, Spark wants to minimize data transfer by executing tasks "as close as possible" to where the data resides. This means that the data and the functions are stored together when possible. 

Spark has several locality levels in the context of Data Locality, ranging from _Data Local_ to _Any_. Spark always try to use the most local level first. If it can not perform the given task within a given time window, it will try again at a less local level. 
[spark.apache.org](https://spark.apache.org/docs/latest/tuning.html#data-locality)

##### Bucketing
To optimize joins, Spark partitions the data into _buckets_. the joins can be performed faster (if bucketing is used properly) because it avoids _shuffling_ of tables that are part of the join. 

[jaceklaskowski.gitbooks.io](https://jaceklaskowski.gitbooks.io/mastering-spark-sql/content/spark-sql-bucketing.html)
##### Distributed Filesystem
A Distributed Filesystem (DFS), is a single filesystem with data stored across several servers. This allows for better, and more cost-efficient, scalability. 

Spark does not have support for DFS on its own, so it is often used on top of the Hadoop Distributed File System(HDFS).

[simplilearn.com](https://www.simplilearn.com/spark-vs-hadoop-article)